In [7]:
import sqlalchemy
import pandas as pd

username = 'postgres'  
password = 'proddb123'
host = '34.84.190.250' 
port = '5432'
database = 'prod' 

db_url = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(
    username, password, host, port, database)

engine = sqlalchemy.create_engine(db_url) #,connect_args={'options': '-c lock_timeout=3000 -c statement_timeout=5000'})
cloud = engine.connect()

In [8]:
def get_prices_crypto_websocket(ticker, bar_size, bars, ohclv=False, return_all_data=False): #this is going to be the one we want to use next, and going to require some further editing 
    #ticker = format_ticker(tick1=ticker)
    table = 'ohlcv_' + str(ticker).lower() + '_' + str(bar_size)
    if return_all_data == False: 
        res = pd.DataFrame(cloud.execute("SELECT close FROM " + str(table) + " ORDER BY timestamp desc").all())#['close'].values.tolist()
        print(res)
        return res[:bars] #results are returned most recent bar first
    else:
        res = pd.DataFrame(cloud.execute("SELECT * FROM " + str(table) + " ORDER BY timestamp desc").all()).values.tolist()
        return res[:bars]
    
def format_ticker(tick1=None, ticker2=None, use_list=False, ticker_list=None):
    if tick1 and not ticker2: 
        if tick1[-4:] == 'USDT':
            return tick1.split('/')[0] + 'USDT'
        elif tick1[-4:] == 'BUSD':
            return tick1.split('/')[0] + 'BUSD'

    if use_list == False: 
        res = []
        if tick1[-4:] == 'USDT':
            res.append(tick1.split('/')[0] + 'USDT')
        elif tick1[-4:] == 'BUSD':
            res.append(tick1.split('/')[0] + 'BUSD')

        if ticker2[-4:] == 'USDT':
            res.append(ticker2.split('/')[0] + 'USDT')
        elif ticker2[-4:] == 'BUSD':
            res.append(ticker2.split('/')[0] + 'BUSD')
        return res[0], res[1]
    elif use_list == True: 
        adj_list = []
        for ticker in ticker_list:
            if ticker[-4:] == 'USDT':
                adj_list.append(ticker.split('/')[0] + 'USDT')
            elif ticker[-4:] == 'BUSD':
                adj_list.append(ticker.split('/')[0] + 'BUSD')
        return adj_list

In [9]:
ticker1 = 'ETHBUSD'
bar_size_lookback='5m'


ticker1_current_price = get_prices_crypto_websocket(ticker1, bar_size_lookback, 1, ohclv=False, return_all_data=True)[0][4]
print(ticker1_current_price)

1566.17
